# Парсинг с помощью BeautifulSoup

Beautiful Soup - это библиотека для Python, которая позволяет парсить (анализировать) HTML и XML документы. Она предоставляет удобный способ искать, навигировать, и модифицировать дерево DOM (Document Object Model), представляющее HTML/XML документ.

# Задание

Вам необходимо собрать датасет, спарсив данные из этого сайта:

https://books.toscrape.com/

Всего на сайте 1000 книг. То есть длина датасета должна равняться количеству книг.
 
Итоговая таблица должна содержать следующие столбцы:

| Название столбца | Описание | 
|--|--|
|id| Идентификатор книги |
|book_name| Название книги |
|price| Цена в £ |
|stock| Наличие книги. 1 или 0|
|url| Ссылка на книгу |

**Примечание по столбцам:**
- `id` - заполняется разработчиком датасета. Первая спарсенная книга имеет `id` = `0`.
- `url` - должна содержать полную ссылку. Не только конец ссылки, указанный на сайте. То есть по данному url можно перейти одним кликом.

## Импорт библиотек

In [1]:
import requests
import pandas as pd

from bs4 import BeautifulSoup
from pprint import pprint

## Cоздание датасета и парсинг данных

In [2]:
from collections import defaultdict


"""
Значение в колонке stock осознано не менял ,
мне не понятно зачем менять понятную запись "In stock"(В наличии) на 1
Если надо могу сделать конечно
"""

def func_parser_one_page(url, soup):
    '''Парсим страницу.'''

    data = defaultdict(list)
    divs = soup.find_all('li', attrs={'class':'col-xs-6 col-sm-4 col-md-3 col-lg-3'})

    for elem in divs:
        data['book_name'].append(elem.find('h3').find('a').get('title'))
        data['price'].append(elem.find('div', class_='product_price').find('p',  class_='price_color').text)
        data['stock'].append(elem.find('div', class_='product_price').find('p', class_='instock availability').text.strip())
        data['url'].append(url + elem.find('h3').find('a').get('href'))

    return data


def paginations(url):
    '''Пагинация.'''

    result = defaultdict(list)
    page = 1
    status_code = 200

    while status_code == 200:

        response = requests.get(url + f"catalogue/page-{page}.html", timeout=10)
        soup = BeautifulSoup(response.content, 'html.parser')
        data_one_page = func_parser_one_page(url, soup)

        for i in  data_one_page:
            result[i] += data_one_page[i]

        page += 1
        status_code = response.status_code

    return result


url = 'https://books.toscrape.com/'

df = pd.DataFrame(paginations(url))
df.index.name = 'id'

display(df)


,book_name,price,stock,url
id,,,,
0,A Light in the Attic,£51.77,In stock,https://books.toscrape.com/a-light-in-the-atti...
1,Tipping the Velvet,£53.74,In stock,https://books.toscrape.com/tipping-the-velvet_...
2,Soumission,£50.10,In stock,https://books.toscrape.com/soumission_998/inde...
3,Sharp Objects,£47.82,In stock,https://books.toscrape.com/sharp-objects_997/i...
4,Sapiens: A Brief History of Humankind,£54.23,In stock,https://books.toscrape.com/sapiens-a-brief-his...
...,...,...,...,...
995,Alice in Wonderland (Alice's Adventures in Won...,£55.53,In stock,https://books.toscrape.com/alice-in-wonderland...
996,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",£57.06,In stock,https://books.toscrape.com/ajin-demi-human-vol...
997,A Spy's Devotion (The Regency Spies of London #1),£16.97,In stock,https://books.toscrape.com/a-spys-devotion-the...


## Итоговый датасет

In [3]:
df.shape

(1000, 4)

In [4]:
display(
    df.head(),
    df.tail()
)

,book_name,price,stock,url
id,,,,
0,A Light in the Attic,£51.77,In stock,https://books.toscrape.com/a-light-in-the-atti...
1,Tipping the Velvet,£53.74,In stock,https://books.toscrape.com/tipping-the-velvet_...
2,Soumission,£50.10,In stock,https://books.toscrape.com/soumission_998/inde...
3,Sharp Objects,£47.82,In stock,https://books.toscrape.com/sharp-objects_997/i...
4,Sapiens: A Brief History of Humankind,£54.23,In stock,https://books.toscrape.com/sapiens-a-brief-his...


,book_name,price,stock,url
id,,,,
995,Alice in Wonderland (Alice's Adventures in Won...,£55.53,In stock,https://books.toscrape.com/alice-in-wonderland...
996,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",£57.06,In stock,https://books.toscrape.com/ajin-demi-human-vol...
997,A Spy's Devotion (The Regency Spies of London #1),£16.97,In stock,https://books.toscrape.com/a-spys-devotion-the...
998,1st to Die (Women's Murder Club #1),£53.98,In stock,https://books.toscrape.com/1st-to-die-womens-m...
999,"1,000 Places to See Before You Die",£26.08,In stock,https://books.toscrape.com/1000-places-to-see-...


# ЗАДАНИЕ ПРО

Так, мы спарсили данные о книгах. Но данные какие-то неполные. Часть названия стирается из-за отображения и нет ни полного названия книги, ни описания этой книги, ни жанра.

Вам необходимо дополнить датасет, спарсив дополнительные данные из того же сайта:

https://books.toscrape.com/
 
Итоговая таблица должна содержать следующие столбцы:

| Название столбца | Описание | 
|--|--|
|id| Идентификатор книги |
|book_name| Название книги - только полное название|
|genre| жанр книги |
|desc| описание |
|price| Цена в £ |
|stock| Наличие книги. 1 или 0|
|url| Ссылка на книгу |
| num_of_rev | количество отзывов|

## Парсинг данных и обогащение датасета

In [5]:
from collections import defaultdict


def get_desc_num_of_rev(url):
    '''Собираем описание и кол-во отзывов'''

    response = requests.get(url, timeout=10)
    soup = BeautifulSoup(response.content, 'html.parser')
    divs = soup.find_all('div', {'class': 'content'})

    lst = []

    for elem in divs:
        lst.append(elem.find('article', {'class': 'product_page'}).find('p', class_=None).text)
        lst.append(elem.find('table', {'class': 'table table-striped'}).find_all('tr')[-1].find('td').text)
    return lst


def get_book_in_genre(url, slag):
    '''Собираем book_name, price, stock, url'''

    response = requests.get(url+slag, timeout=10)
    soup = BeautifulSoup(response.content, 'html.parser')
    divs = soup.find_all('div', {'class': 'col-sm-8 col-md-9'})

    data = {}

    for elem in divs:
        data['book_name'] = elem.find('section').find('h3').find('a').get('title')
        data['price'] = elem.find('section').find('div', {'class': 'product_price'}).find('p').text
        data['stock'] = elem.find('section').find('div', {'class': 'product_price'}).find('p', class_='instock availability').text.strip()
        data['url'] = (url + 'catalogue/' + elem.find('section').find('h3').find('a').get('href').lstrip('../'))
        data['desc'] = get_desc_num_of_rev(data['url'])[0]
        data['num_of_rev'] = get_desc_num_of_rev(data['url'])[1]

    return data


def func_parser_one_page(url, soup, df):
    '''Парсим страницу.'''

    data = {}
    divs = soup.find_all('div', attrs={'class':'side_categories'})

    for elem in divs:
        slag = elem.find('ul').find('li').find('ul').find('a').get('href')
        data['genre'] = elem.find('ul').find('li').find('ul').find('li').find('a').text.split('\n')[2].strip()
        data.update(get_book_in_genre(url, slag))

        df.loc[len(df)] = data

    return df


def paginations_in_gengre(url, soup, df):
    '''Пагинация.'''

    # Нужно еще добавить пагинацию !!!
    pass


df = pd.DataFrame({
    'book_name':[],
    'genre':[],
    'desc':[],
    'price':[],
    'stock':[],
    'url':[],
    'num_of_rev':[],
})

url = 'https://books.toscrape.com/'
response = requests.get(url, verify=False, timeout=10)
soup = BeautifulSoup(response.content, 'html.parser')

display(func_parser_one_page(url, soup, df))


c:\Dev\1T_data_analyst\venv\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'books.toscrape.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


,book_name,genre,desc,price,stock,url,num_of_rev
0,It's Only the Himalayas,Travel,"“Wherever you go, whatever you do, just . . . ...",£45.17,In stock,https://books.toscrape.com/catalogue/its-only-...,0


## Итоговый датасет PRO

In [6]:
df.shape

(1, 7)

In [7]:
display(
    df.head(),
    df.tail()
)

,book_name,genre,desc,price,stock,url,num_of_rev
0,It's Only the Himalayas,Travel,"“Wherever you go, whatever you do, just . . . ...",£45.17,In stock,https://books.toscrape.com/catalogue/its-only-...,0


,book_name,genre,desc,price,stock,url,num_of_rev
0,It's Only the Himalayas,Travel,"“Wherever you go, whatever you do, just . . . ...",£45.17,In stock,https://books.toscrape.com/catalogue/its-only-...,0
